## Dbt 실행 후, seed_dev 및 gold_dev 스키마의 테이블 조회
커널 선택: http://4.230.26.44:8888

### Spark session 생성

In [1]:
import os
import sys

sys.path.append("/app")
from src.utils.spark_builder import get_spark_session
from pyspark.sql import functions as F

os.environ[
    "PYSPARK_SUBMIT_ARGS"
] = """
    --conf spark.cores.max=2
    --conf spark.executor.memory=8g
    --conf spark.app.name=GDELT_dbt_test
    pyspark-shell
"""

# Spark 세션 생성
spark = get_spark_session("GDELT_dbt_test", "spark://spark-master:7077")
print("✅ Spark 세션 생성 완료")

25/09/28 08:42:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


✅ Spark 세션 생성 완료


In [4]:
# Spark Metastore에 존재하는 모든 데이터베이스(스키마) 목록을 확인합니다.
print("📖 사용 가능한 데이터베이스(스키마) 목록:")
spark.sql("SHOW DATABASES").show()

📖 사용 가능한 데이터베이스(스키마) 목록:
+------------+
|   namespace|
+------------+
|     default|
|    gold_dev|
|   gold_prod|
|    seed_dev|
|   seed_prod|
|      silver|
| staging_dev|
|staging_prod|
+------------+



In [5]:
# dbt seed 또는 dbt run으로 생성된 테이블 확인
print("📜 '_dev' 스키마의 테이블 목록:")
spark.sql("SHOW TABLES IN seed_dev").show()
spark.sql("SHOW TABLES IN staging_dev").show()
spark.sql("SHOW TABLES IN gold_dev").show()

📜 '_dev' 스키마의 테이블 목록:
+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
| seed_dev|actor_ethnic_grou...|      false|
| seed_dev|actor_organizatio...|      false|
| seed_dev|actor_religion_codes|      false|
| seed_dev|    actor_role_codes|      false|
| seed_dev|  event_detail_codes|      false|
| seed_dev|event_quad_class_...|      false|
| seed_dev|    event_root_codes|      false|
| seed_dev|       geo_adm_codes|      false|
| seed_dev|   geo_country_codes|      false|
| seed_dev|      geo_type_codes|      false|
+---------+--------------------+-----------+

+-----------+--------------------+-----------+
|  namespace|           tableName|isTemporary|
+-----------+--------------------+-----------+
|staging_dev|stg_actors_descri...|      false|
|staging_dev|stg_gkg_detailed_...|      false|
|staging_dev|    stg_seed_mapping|      false|
+-----------+--------------------+-----------+

+---------+------

### Gold_dev의 모든 테이블 크기 확인용

In [3]:
schema_name = "gold_dev"

try:
    tables_df = spark.sql(f"SHOW TABLES IN {schema_name}")
    table_names = [row.tableName for row in tables_df.collect()]

    if not table_names:
        print(f"❌ '{schema_name}' 스키마에서 테이블을 찾을 수 없습니다.")
    else:
        print(
            f"🥇 '{schema_name}' 스키마에서 총 {len(table_names)}개의 테이블을 발견했습니다.\n"
        )

    for table in table_names:
        full_table_name = f"{schema_name}.{table}"
        print(f"테이블명: {full_table_name}")

        try:
            df = spark.table(full_table_name)
            row_count = df.count()
            col_count = len(df.columns)
            print(f" - 행(Row)의 수: {row_count} 개")
            print(f" - 열(Column)의 수: {col_count} 개\n")

        except Exception as e:
            print(f"❌ 테이블 '{full_table_name}' 조회 중 오류 발생: {e}\n")

except Exception as e:
    print(f"❌ 스키마 '{schema_name}'의 테이블 목록을 가져오는 중 오류 발생: {e}")

🥇 'gold_dev' 스키마에서 총 8개의 테이블을 발견했습니다.

테이블명: gold_dev.gold_bigq_chart_weekday_event


25/09/27 15:18:48 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/09/27 15:18:55 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


 - 행(Row)의 수: 3468 개
 - 열(Column)의 수: 5 개

테이블명: gold_dev.gold_chart_events_category


 - 행(Row)의 수: 14016 개
 - 열(Column)의 수: 8 개

테이블명: gold_dev.gold_chart_events_count_avgtone


 - 행(Row)의 수: 1890 개
 - 열(Column)의 수: 7 개

테이블명: gold_dev.gold_chart_weekday_event_ratio


 - 행(Row)의 수: 2311 개
 - 열(Column)의 수: 7 개

테이블명: gold_dev.gold_daily_actor_network


 - 행(Row)의 수: 1709 개
 - 열(Column)의 수: 17 개

테이블명: gold_dev.gold_daily_detailed_events


 - 행(Row)의 수: 459392 개
 - 열(Column)의 수: 81 개

테이블명: gold_dev.gold_daily_rich_story


 - 행(Row)의 수: 1049 개
 - 열(Column)의 수: 11 개

테이블명: gold_dev.gold_near_realtime_summary


 - 행(Row)의 수: 1890 개
 - 열(Column)의 수: 16 개



### Seed 확인 및 테이블 관리

In [5]:
# Seed 테이블(총 10개)의 샘플 데이터를 5개씩 출력하여 내용 확인
seed_tables = [
    "actor_ethnic_group_codes",
    "actor_organization_codes",
    "actor_religion_codes",
    "actor_role_codes",
    "event_root_codes",
    "event_detail_codes",
    "event_quad_class_codes",
    "geo_adm_codes",
    "geo_country_codes",
    "geo_type_codes",
]

for table in seed_tables:
    print(f"\n 🔍 {table} 테이블 샘플 데이터")
    try:
        spark.sql(f"SELECT * FROM seed_dev.{table} LIMIT 5").show(truncate=False)
    except Exception as e:
        print(f"❌ 테이블 조회 중, 오류 발생: {e}")


 🔍 actor_ethnic_group_codes 테이블 샘플 데이터


25/09/26 16:58:47 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


+----+---------------------+
|code|description          |
+----+---------------------+
|aar |아파르족             |
|abk |압하스인             |
|abr |오스트레일리아 원주민|
|ace |아체인               |
|acg |아창족               |
+----+---------------------+


 🔍 actor_organization_codes 테이블 샘플 데이터
+----+----+--------------------------+
|code|type|description               |
+----+----+--------------------------+
|BHF |정부|보스니아 헤르체고비나 연방|
|FTA |정부|파타                      |
|HAM |정부|하마스                    |
|RPF |정부|르완다 애국 전선          |
|SRP |정부|스릅스카 공화국           |
+----+----+--------------------------+


 🔍 actor_religion_codes 테이블 샘플 데이터
+----+----+---------------+
|code|type|description    |
+----+----+---------------+
|REL |일반|종교 (미지정)  |
|ATH |이념|불가지론/무신론|
|BAH |종교|바하이 신앙    |
|BUD |종교|불교           |
|MAH |종파|대승불교       |
+----+----+---------------+


 🔍 actor_role_codes 테이블 샘플 데이터
+----+--------+--------------+
|code|type    |description   |
+----+--------+--------------+
|COP |1차 역할|경찰          |
|GOV

생성된 테이블을 삭제할 경우, 아래 코드에 해당 스키마와 "삭제할 테이블명"을 수정하여 실행

In [ ]:
tables_to_drop = [
    # "[삭제할 테이블명]"
]
print("🗑️ 기존 테이블 삭제를 시작합니다.")

for table_name in tables_to_drop:
    try:
        spark.sql(f"DROP TABLE IF EXISTS gold_dev.{table_name}")
        print(f"  - 테이블 'gold_dev.{table_name}' 삭제 완료.")
    except Exception as e:
        print(f"  - 테이블 'gold_dev.{table_name}' 삭제 중 오류 발생: {e}")

print("\n✅ 테이블 삭제 작업이 완료되었습니다.")

# 삭제 후 테이블 목록을 다시 확인하여 깨끗해졌는지 확인
print("📜 현재 'gold_dev' 스키마의 테이블 목록:")
spark.sql("SHOW TABLES IN gold_dev").show()

### Staging 폴더의 SQL 파일을 dbt run한 결과 확인

In [7]:
print("🥈 [Staging] stg_seed_mapping 테이블 조회 결과")
spark.sql(
    "SELECT * FROM staging_dev.stg_seed_mapping ORDER BY event_date DESC LIMIT 20"
).show(truncate=False)

🥈 [Staging] stg_seed_mapping 테이블 조회 결과


+---------------+----------+-------------+-------------------+------------------------+----------+-------------+---------------+------------+-----------+------------+-----------------+-----------+--------------+--------------------------+--------------------------------------------------------+--------------------------+----------------------+---------------------------+----------------+------------------+-----------------------+--------------+-------------------+------------------+------------------------------------+-------------------------+--------------------------------------------------------+-------------------------+--------------+---------------+-----------+----------------+--------------------------+--------------------------------------------------------+--------------------------+----------------------+---------------------------+----------------+------------------+-----------------------+--------------+-------------------+------------------+-------------------------------

In [8]:
print("🥈 [Staging] stg_actors_description 테이블 조회 결과")
spark.sql(
    "SELECT * FROM staging_dev.stg_actors_description WHERE actor_full_description IS NULL"
).show(truncate=False)

🥈 [Staging] stg_actors_description 테이블 조회 결과


+----------+----------------------+
|actor_code|actor_full_description|
+----------+----------------------+
+----------+----------------------+



In [9]:
print("🥈 [Staging] stg_gkg_detailed_events 테이블 조회 결과")
spark.sql(
    "SELECT * FROM staging_dev.stg_gkg_detailed_events ORDER BY event_date DESC LIMIT 20"
).show(truncate=False)

🥈 [Staging] stg_gkg_detailed_events 테이블 조회 결과


+---------------+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+-----------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Marts 폴더의 SQL 파일을 dbt run한 결과 확인

In [10]:
print("🥇 [Marts] gold_near_realtime_summary 테이블 조회 결과")
spark.sql(
    "SELECT * FROM gold_dev.gold_near_realtime_summary ORDER BY event_date ASC"
).show(truncate=False)

🥇 [Marts] gold_near_realtime_summary 테이블 조회 결과
+----------+-------------------------+-------------------------+-------------------------+-----------+-------------------+--------------------+--------------+-------------+--------------+-----------------------+--------------------+-------------------+--------------------+-------------------+-------------------------------------------------------------------------------------------+
|event_date|mp_action_geo_country_iso|mp_action_geo_country_eng|mp_action_geo_country_kor|event_count|avg_goldstein_scale|avg_tone            |total_mentions|total_sources|total_articles|count_cooperation_event|count_conflict_event|count_anomaly_event|risk_score_daily    |processed_at       |daily_tone_summary                                                                         |
+----------+-------------------------+-------------------------+-------------------------+-----------+-------------------+--------------------+--------------+-------------+---------

In [ ]:
print("🥇 [Marts] gold_daily_detailed_events 테이블 조회 결과")
spark.sql(
    "SELECT * FROM gold_dev.gold_daily_detailed_events WHERE amounts IS NOT NULL"
).show(truncate=False)

🥇 [Marts] gold_daily_detailed_events 테이블 조회 결과


+---------------+----------+-------------+-------------------+--------------+----------+-------------+---------------+------------+-----------+------------+-----------------+-----------+--------------------+--------------------------+--------------------------+--------------------------+----------------------+---------------------------+----------------+------------------+-----------------------+--------------+-------------------+------------------+---------------------------------+-------------------------+----------------------------+-------------------------+--------------+---------------+-----------+--------------------+--------------------------+--------------------------------------------------------+--------------------------+----------------------+---------------------------+----------------+------------------+-----------------------+--------------+-------------------+------------------+-------------------------------+-----------------------+-------------------------+----------

In [5]:
print("🥇 [Marts] gold_daily_rich_story 테이블 조회 결과")
spark.sql(
    "SELECT * FROM gold_dev.gold_daily_rich_story ORDER BY mp_action_geo_country_iso ASC, event_date ASC"
).show(truncate=False)

🥇 [Marts] gold_daily_rich_story 테이블 조회 결과


25/09/28 09:08:47 WARN DeltaLog: Change in the table id detected while updating snapshot. 
Previous snapshot = Snapshot(path=s3a://warehouse/gold_dev/gold_daily_rich_story/_delta_log, version=12, metadata=Metadata(069dbba3-0395-4036-953e-95ab0277e82a,null,null,Format(parquet,Map()),{"type":"struct","fields":[{"name":"event_date","type":"date","nullable":true,"metadata":{}},{"name":"mp_action_geo_country_iso","type":"string","nullable":true,"metadata":{}},{"name":"mp_action_geo_country_eng","type":"string","nullable":true,"metadata":{}},{"name":"mp_action_geo_country_kor","type":"string","nullable":true,"metadata":{}},{"name":"story_type","type":"string","nullable":true,"metadata":{}},{"name":"rich_story","type":"string","nullable":true,"metadata":{}},{"name":"goldstein_scale","type":"double","nullable":true,"metadata":{}},{"name":"avg_tone","type":"double","nullable":true,"metadata":{}},{"name":"event_type","type":"string","nullable":true,"metadata":{}},{"name":"source_url","type":"str

+----------+-------------------------+-------------------------+-------------------------+-----------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+-----------------+----------+-------------------------------------------------------------------------------------------------------------------------------------------+-------------------+
|event_date|mp_action_geo_country_iso|mp_action_geo_country_eng|mp_action_geo_country_kor|story_type             |rich_story                                                                                                                                                                                                                             

In [12]:
print("🥇 [Marts] gold_daily_actor_network 테이블 조회 결과")
spark.sql(
    "SELECT * FROM gold_dev.gold_daily_actor_network ORDER BY event_date ASC"
).show(truncate=False)

🥇 [Marts] gold_daily_actor_network 테이블 조회 결과


+----------+------------------+--------------------------------------------------------+----------------------+--------------+---------------+------------------+--------------------------------------------------------+------------------+--------------+---------------+-----------+-----------------+-------------------+-----------------------------------------------+----------+-------------+
|event_date|actor1_country_iso|actor1_country_eng                                      |actor1_country_kor    |actor1_geo_lat|actor1_geo_long|actor2_country_iso|actor2_country_eng                                      |actor2_country_kor|actor2_geo_lat|actor2_geo_long|event_count|avg_tone         |processed_at       |location_text                                  |route_name|tone_category|
+----------+------------------+--------------------------------------------------------+----------------------+--------------+---------------+------------------+-------------------------------------------------------

In [13]:
print("🥇 [Marts] gold_chart_events_category 테이블 조회 결과")
spark.sql(
    "SELECT * FROM gold_dev.gold_chart_events_category ORDER BY event_date ASC"
).show(truncate=False)

🥇 [Marts] gold_chart_events_category 테이블 조회 결과


+----------+-------------------------+--------------------------------------------------------+--------------------------+-------------+-------------------+-----------+-------------------+
|event_date|mp_action_geo_country_iso|mp_action_geo_country_eng                               |mp_action_geo_country_kor |mp_quad_class|mp_event_categories|event_count|processed_at       |
+----------+-------------------------+--------------------------------------------------------+--------------------------+-------------+-------------------+-----------+-------------------+
|2024-09-23|PHL                      |The Philippines                                         |필리핀                    |언어적 협력  |호소 및 요청       |1          |2025-09-23 13:15:00|
|2024-09-23|USA                      |The United States of America                            |미국                      |언어적 협력  |협의               |53         |2025-09-23 23:45:00|
|2024-09-23|KEN                      |Kenya                                  

In [14]:
print("🥇 [Marts] gold_chart_events_count_avgtone 테이블 조회 결과")
spark.sql(
    "SELECT * FROM gold_dev.gold_chart_events_count_avgtone ORDER BY event_date ASC"
).show(truncate=False)

🥇 [Marts] gold_chart_events_count_avgtone 테이블 조회 결과
+----------+-------------------------+--------------------------------------------------------+--------------------------+-----------+--------------------+-------------------+
|event_date|mp_action_geo_country_iso|mp_action_geo_country_eng                               |mp_action_geo_country_kor |event_count|avg_tone            |processed_at       |
+----------+-------------------------+--------------------------------------------------------+--------------------------+-----------+--------------------+-------------------+
|2024-09-23|AUS                      |Australia                                               |호주                      |27         |-2.512694532501507  |2025-09-23 22:30:00|
|2024-09-23|LAO                      |Lao People's Democratic Republic                        |라오 인민 민주 공화국     |1          |-3.81679389312977   |2025-09-23 09:45:00|
|2024-09-23|GBR                      |The United Kingdom of Great Britain and N

In [15]:
print("🥇 [Marts] gold_chart_weekday_event_ratio 테이블 조회 결과")
spark.sql(
    "SELECT * FROM gold_dev.gold_chart_weekday_event_ratio ORDER BY period ASC"
).show(truncate=False)

🥇 [Marts] gold_chart_weekday_event_ratio 테이블 조회 결과
+------------------+-------------------------+-------------------------+-------------------------+-----------+------------+------------------+
|period            |mp_action_geo_country_iso|mp_action_geo_country_eng|mp_action_geo_country_kor|weekday_num|weekday_name|event_percentage  |
+------------------+-------------------------+-------------------------+-------------------------+-----------+------------+------------------+
|이번 달 (25년 9월)|ABW                      |Aruba                    |아루바                   |5          |Thu         |15.38461538461538 |
|이번 달 (25년 9월)|ABW                      |Aruba                    |아루바                   |4          |Wed         |34.61538461538462 |
|이번 달 (25년 9월)|ABW                      |Aruba                    |아루바                   |3          |Tue         |38.46153846153846 |
|이번 달 (25년 9월)|ABW                      |Aruba                    |아루바                   |6          |Fri         |

### Spark 세션 종료

In [1]:
spark.stop()
print("\n✅ Spark 세션이 종료되었습니다.")

NameError: name 'spark' is not defined